In [1]:
import pandas as pd
from db import DB

In [2]:
db = DB()

In [3]:
start_date = '2023-09-11 00:00:00'
funnel_steps_logs = db.get_funnel_steps_log(start_date)
user_data = db.get_user_data(start_date)
cart_data = db.get_cart_data(start_date)

In [4]:
time_on_page = funnel_steps_logs.copy()
# funnel_steps_logs.drop('Unnamed: 0', axis=1)
time_on_page['page'] = time_on_page.groupby('cart_id')['funnel_step'].shift(1)
time_on_page['prev_changed_at'] = time_on_page.groupby('cart_id')['changed_at'].shift(1)
time_on_page['minutes_spent'] = (time_on_page['changed_at'] - time_on_page['prev_changed_at']).dt.total_seconds() / 60

In [5]:
time_on_page

,cart_id,changed_at,funnel_step,page,prev_changed_at,minutes_spent
1087631,24037193,2023-09-12 12:51:43,"""shipping""",NaN,NaT,NaN
1063978,24037193,2023-09-12 13:17:58,"""cart""","""shipping""",2023-09-12 12:51:43,26.250000
1063714,24037193,2023-09-12 13:18:15,"""shipping""","""cart""",2023-09-12 13:17:58,0.283333
1059899,24037193,2023-09-12 13:22:21,"""cart""","""shipping""",2023-09-12 13:18:15,4.100000
1057540,24037193,2023-09-12 13:24:50,"""shipping""","""cart""",2023-09-12 13:22:21,2.483333
...,...,...,...,...,...,...
26,350699902,2023-09-13 17:40:11,"""cart""",NaN,NaT,NaN
18,350699903,2023-09-13 17:40:13,"""cart""",NaN,NaT,NaN
13,350699904,2023-09-13 17:40:13,"""cart""",NaN,NaT,NaN
9,350699905,2023-09-13 17:40:13,"""cart""",NaN,NaT,NaN


In [6]:
platforms = cart_data.copy()


def platform(source: str):
    if source == 'web' or source == 'mobile':
        return 'web'
    return 'app'


platforms['platform'] = platforms['source'].apply(lambda x: platform(x))
platforms['closed_platform'] = platforms['source_close'].apply(lambda x: platform(x))
df = pd.merge(left=platforms, right=time_on_page, left_on='id', right_on='cart_id')

In [7]:
df

,id,status,payable_price,source,source_close,platform,closed_platform,cart_id,changed_at,funnel_step,page,prev_changed_at,minutes_spent
0,350285502,open,0,android,None,app,app,350285502,2023-09-11 00:00:01,"""cart""",NaN,NaT,NaN
1,350285503,closed,19065700,android,android,app,app,350285503,2023-09-11 00:00:02,"""cart""",NaN,NaT,NaN
2,350285503,closed,19065700,android,android,app,app,350285503,2023-09-11 18:39:42,"""shipping""","""cart""",2023-09-11 00:00:02,1119.666667
3,350285503,closed,19065700,android,android,app,app,350285503,2023-09-11 18:39:54,"""cart""","""shipping""",2023-09-11 18:39:42,0.200000
4,350285503,closed,19065700,android,android,app,app,350285503,2023-09-11 18:43:51,"""shipping""","""cart""",2023-09-11 18:39:54,3.950000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042078,350690713,open,0,android,None,app,app,350690713,2023-09-13 16:30:07,"""cart""",NaN,NaT,NaN
1042079,350690713,open,0,android,None,app,app,350690713,2023-09-13 16:30:14,"""shipping""","""cart""",2023-09-13 16:30:07,0.116667
1042080,350690713,open,0,android,None,app,app,350690713,2023-09-13 16:30:26,"""payment""","""shipping""",2023-09-13 16:30:14,0.200000
1042081,350690713,open,0,android,None,app,app,350690713,2023-09-13 16:30:29,"""checkout""","""payment""",2023-09-13 16:30:26,0.050000


In [8]:
df['churned'] = (df['minutes_spent'].isnull()) | (df['minutes_spent'] > 45)

In [9]:
df

,id,status,payable_price,source,source_close,platform,closed_platform,cart_id,changed_at,funnel_step,page,prev_changed_at,minutes_spent,churned
0,350285502,open,0,android,None,app,app,350285502,2023-09-11 00:00:01,"""cart""",NaN,NaT,NaN,True
1,350285503,closed,19065700,android,android,app,app,350285503,2023-09-11 00:00:02,"""cart""",NaN,NaT,NaN,True
2,350285503,closed,19065700,android,android,app,app,350285503,2023-09-11 18:39:42,"""shipping""","""cart""",2023-09-11 00:00:02,1119.666667,True
3,350285503,closed,19065700,android,android,app,app,350285503,2023-09-11 18:39:54,"""cart""","""shipping""",2023-09-11 18:39:42,0.200000,False
4,350285503,closed,19065700,android,android,app,app,350285503,2023-09-11 18:43:51,"""shipping""","""cart""",2023-09-11 18:39:54,3.950000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042078,350690713,open,0,android,None,app,app,350690713,2023-09-13 16:30:07,"""cart""",NaN,NaT,NaN,True
1042079,350690713,open,0,android,None,app,app,350690713,2023-09-13 16:30:14,"""shipping""","""cart""",2023-09-13 16:30:07,0.116667,False
1042080,350690713,open,0,android,None,app,app,350690713,2023-09-13 16:30:26,"""payment""","""shipping""",2023-09-13 16:30:14,0.200000,False
1042081,350690713,open,0,android,None,app,app,350690713,2023-09-13 16:30:29,"""checkout""","""payment""",2023-09-13 16:30:26,0.050000,False


In [10]:
df['flow'] = df['page'].astype(str) + '->' + df['funnel_step'].astype(str)

In [11]:
grouped = df.groupby(["flow", "churned", "source", 'platform'])['cart_id'].count().reset_index()
grouped.rename(columns={"cart_id": "count"}, inplace=True)

In [12]:
grouped

,flow,churned,source,platform,count
0,"""cart""->""cart""",False,android,app,1096
1,"""cart""->""cart""",False,ios,app,28
2,"""cart""->""cart""",False,mobile,web,99
3,"""cart""->""cart""",False,web,web,190
4,"""cart""->""cart""",True,android,app,113
...,...,...,...,...,...
101,"nan->""checkout""",True,android,app,17
102,"nan->""checkout""",True,web,web,7
103,"nan->""payment""",True,android,app,1
104,"nan->""shipping""",True,android,app,16
